In [1]:
(ns groud-up.ch6)

nil

### Immutability
- the references we've used in `let` bindings and function arguments are immutable: they never change
- Functions remember the values of those symbols at the time the function was constructed

In [2]:
(defn present
    [gift]
    (fn [] gift))

#'groud-up.ch6/present

In [3]:
(def green-box (present "clockwork beetle"))

#'groud-up.ch6/green-box

In [4]:
(def red-box (present "plush tiger"))

#'groud-up.ch6/red-box

In [5]:
(red-box)

"plush tiger"

In [6]:
(green-box)

"clockwork beetle"

- the `present` function returns a new function
- That function takes no arguemnts and always return the gift.
- Because `gift` if not an argument ot the inner function, it refers to the value from the outer function body
- When we packaged up the red and green boxes, the functions we created carried with them a memory of the `gift` symbol's value (which is in enclosing scopes) (even if the value are not present in memory.)
- This is called __closing over__ the `gift` variable.
- The inner function si called a __closure__. 
- In Clojure, new functions close over _all_ variables except their arguments . 

### Delays
- Because functions close over variables, they can be used to _defer_ evaluation of expressions

In [7]:
(do (prn "Adding") (+ 1 2))

"Adding"


3

In [8]:
(def later (fn [] (prn "Adding") (+ 1 2)))

#'groud-up.ch6/later

In [9]:
(later)

"Adding"


3

- Evaluating `(def later ...)` did not evaluate the expressions in the function body. Only when we invoked the function `later` did Clojure print strings and return a value. 
- This is the basis of _concurrency_, evaluating expressions outside their normal, sequential order
- Deferrign evaluation is so common that there's a standard macro for it, called `delay`.

In [10]:
(def later (delay (prn "Adding") (+ 1 2)))

#'groud-up.ch6/later

In [11]:
later

#delay[{:status :pending, :val nil} 0xadefa90]

In [12]:
(deref later)

"Adding"


3

In [13]:
(deref later) ;; notice that nothing prints out!!!

3

- Instead of a function, `delay` creates a special type of Delay object: an identity which refers to those expressions which should be evaluated later. 
- We extract, or _dereference_, the value of that identity with `deref`. 
- `Delay` follows the same rule as functions, closing over lexical scope - because `delay` actually macroexpands into an anonymous function

In [14]:
(clojure.repl/source delay)

(defmacro delay
  "Takes a body of expressions and yields a Delay object that will
  invoke the body only the first time it is forced (with force or deref/@), and
  will cache the result and return it on all subsequent force
  calls. See also - realized?"
  {:added "1.0"}
  [& body]
    (list 'new 'clojure.lang.Delay (list* `^{:once true} fn* [] body)))


nil

Why `Delay` object instead of just a function?
- Since delays only evaluate their expressions once. They remember their value after the first evaluation and return ir for every successive `deref`

In [15]:
(deref later)  ;; nothing prints out!!!

3

In [16]:
@later ;; shortcut for deref!!!

3

### Future
- what if we wanted to opportunistically defer computation?
- `future` is a delay which is evaluated in parallel. 

In [17]:
(def x (future (prn "hi") (+ 1 2)))

#'groud-up.ch6/x

In [18]:
x

#future[{:status :ready, :val 3} 0x4e263805]

In [19]:
(deref x)

3

- `"hi"` is printed right away because futures are evaluated in a new thread.
- On multicore computers, two threads can run in parallel, on different cores at the same time. 
- When there are more threads than cores, the cores trade off running different threads. 
- Both parallel and non-parallel evaluation of threads are concurrent because expressions from different threads can be evaluated out of order

In [20]:
;; repeatedly executes body (presumably for side-effects)
(dotimes [i 5] (prn i))

0
1
2
3
4


nil

In [21]:
;; we use future to run the evaluations in parallel
(dotimes [i 5] (future (prn i)))

01

2
4

nil

- There are techniques to control this concurrent execution so that things happen in some well-defined sequence, like agents and locks
- just like delays, we can deref a future as many times as we want, and the expressions are only evaluated once

In [22]:
(def x (future (prn "hi") (+ 1 2)))

"hi"


#'groud-up.ch6/x

In [23]:
@x

3

In [24]:
@x

3

futures are the most generic parallel construct in Clojure, you can use futures to
- do CPU-intensive computation faster
- wait for multiple network requests to complete at once
- run housekeeping code periodically

### Promises
we want to defer something we don't even have yet....

In [25]:
(def box (promise))

#'groud-up.ch6/box

In [26]:
@box

*Interrupted*


interrupted: 

In [27]:
(deliver box :live-scorpions!)

#promise[{:status :ready, :val :live-scorpions!} 0x7585fe6b]

In [28]:
@box

:live-scorpions!

In [29]:
(deliver box :puppy)

nil

In [30]:
@box

:live-scorpions!

- Like delays and futures, once evaluated, promise always refers to the same value
- This is a simple identity type: we can set it to a value once and read it as many times as we want
- `Promise` is also a concurrency primitive: it guarantees that any attempt to ead the value will wait until the value has been written. 
- We can use promises to synchronize a program which is being evaluated concurrently

In [31]:
(def card (promise))

#'groud-up.ch6/card

In [32]:
(def dealer (future
                (Thread/sleep 5000)
                (deliver card [(inc (rand-int 13))
                               (rand-nth [:clubs :spades :hearts :diamonds])])))

#'groud-up.ch6/dealer

In [33]:
@card

[4 :clubs]

- where delays are lazy, and futures are parallel, promises are concurrent without specifying how the evaluation occurs. 
- think of both delays and futures as being built atop promises, in a way

### Vars
refer to different values at different points in time

In [34]:
(def x :mouse)

#'groud-up.ch6/x

In [35]:
(def box (fn [] x))

#'groud-up.ch6/box

In [36]:
(box)

:mouse

In [37]:
(def x :cat)

#'groud-up.ch6/x

In [38]:
(box)

:cat

- the `box` function closed over `x`, but calling `(box)` returned different results depending on the current value of `x`. 
- Even though the var `x` remained unchanged throughout this example, the value associated with that var did change!
- A reference which is the same everywhere is called a _global variable_ or simply a _global_
- with a __dynamics var__, we can override their value only within the scope of a particular function call, and nowhere else

In [39]:
(def ^:dynamic *board* :maple) ;; by convention, we use asterisks to define dynamic vars

#'groud-up.ch6/*board*

In [40]:
(defn cut [] (prn "sawing through" *board*))

#'groud-up.ch6/cut

- `cut` closes over the var `*board*`, but not the value!
- every time the function is invoked, it looks up the _current_ value of `*board*`

In [41]:
(cut)

"sawing through" :maple


nil

In [42]:
(def *board* "can I do this?")

#'groud-up.ch6/*board*

In [43]:
(cut)

"sawing through" "can I do this?"


nil

In [44]:
(binding [*board* :cedar] (cut))

Execution error (IllegalStateException) at groud-up.ch6/eval4175 (REPL:1).
Can't dynamically bind non-dynamic var: groud-up.ch6/*board*


class java.lang.IllegalStateException: 

In [45]:
;; let's start over again
(def ^:dynamic *board* :maple)

#'groud-up.ch6/*board*

In [46]:
(cut)

"sawing through" :maple


nil

In [47]:
(binding [*board* :cedar] (cut))

"sawing through" :cedar


nil

In [48]:
(cut)

"sawing through" :maple


nil

- `let` and `fn` create immutable lexical scope, but `binding` creates _dynamics scope_
- Lexical scope is constrained to the literal text of the `fn` or `let` expression
- Dynamics scope propagates through function calls!
- This safety property holds even when the program is executed in multiple threads: only the thread which evaluated the binding expression uses that value (specified in binding). Other threads are unaffected. 

- In real programs, you should only mutate vars sparingly. They are intended for naming functions, important bits of global data, and for tracking the environment of a program.
- Don't use vars for mutable program state

### Atoms

In [49]:
;; building up a set of integers
(def xs #{})
(dotimes [i 10] (def xs (conj xs i)))
xs

#{0 7 1 4 6 3 2 9 5 8}

- Vars can be read, set and dynamically bound, but they aren't easy to evolve. 
- In the above example, we use the common paradigm as in imperative languages, where all variables are mutable by default
- However, the program is not __thread safe__

In [50]:
(def xs #{})
(dotimes [i 10] (future (def xs (conj xs i))))
xs

#{0 7 1 4 6 3 2 9 5 8}

- The read-modify-update process assumed that all updates would be consecutive - not concurrent
- When we ask the program to do two read-modify-updates at the same time, updates were lost
- We need an identity which supports safe transformation from one state to another - __atoms__!

In [51]:
(def xs (atom #{}))

#'groud-up.ch6/xs

In [52]:
xs

#atom[#{} 0x293d3d5]

- Unlike vars, atoms are not transparent.
- When evaluated, they don't return their underlying values, but notice that when printed, the current value is hiding inside. 
- To get the current value out of an atom, we have to use `deref` or `@`

In [53]:
@xs

#{}

- Like vars, atoms can be set to a particular value, but instead of `def`, we use `reset!`
- The exclamation point (called a _bang_) is there to remind us that this function modifies the state of its argument - in this case, changing the value of the atom.

In [54]:
(reset! xs :foo)

:foo

In [55]:
xs

#atom[:foo 0x293d3d5]

In [56]:
@xs

:foo

- Unlike vars, atoms can be safely updated using `swap!`
- `swap!` uses a pure function which takes the current value of the atom and returns a **new** value. 
- Under the hood, Clojure does some tricks to make sure these updates are **linearizable**
    - All updates with `swap!` complete in what appears to be in a single consecutive order
    - The effect of a swap! never takes place before calling `swap!`
    - The effect of a swap! is visible to everyone once `swap!` returns
   
hmmmm.... what do these actually mean?

In [57]:
(def x (atom 0))

#'groud-up.ch6/x

In [58]:
(swap! x inc) ;; swap reads value of 0, increase to 1 and writes 1 back to the atom

1

In [59]:
@x

1

In [60]:
(swap! x inc) ;; each call to swap! returns the value that was just written

2

In [61]:
@x

2

In [62]:
;; additional arguments will be placed at the end
(swap! x + 5 6) ;; (+ x 5 6)

13

In [63]:
(def xs (atom #{}))
(dotimes [i 10] (future (swap! xs conj i)))
xs

#atom[#{0 7 1 4 6 3 2 9 5 8} 0x1887785d]

In [64]:
@xs ;; the order of the set is arbitrary, the key is that we didn't miss any number

#{0 7 1 4 6 3 2 9 5 8}

- the function we use to update an atom must be __pure__ - must not mutate any state.
- Clojure's reliance on immutable datatypes, immutable variables and pure functions enables this approach to linearizable mutability. 
- Reach for an atom whenever you want to update a single thing over time

### Ref
- atoms are only linearizable individually.
- for multi-identity updates, we need a stronger safety property than single-atom linearizability - __serializability__ - a global order. Enter `ref`

In [65]:
(def x (ref 0))

#'groud-up.ch6/x

In [66]:
@x

0

- refs are updated in groups using `dosync` transactions 
- we can update refs using `alter` instead of `swap!`
- we can set refs to new values using `ref-set`. instead of `reset!`

In [67]:
(def x (ref 0))
(def y (ref 0))
(dosync
 (ref-set x 1)
 (ref-set y 2))

2

In [68]:
[@x @y]

[1 2]

In [69]:
(def x (ref 0))
(def y (ref 0))
(dosync
 (alter x + 2)
 (alter y inc))

1

In [70]:
[@x @y]

[2 1]

- All `alter` operations within a `dosync` take place atomically, their effects are never interleaved with other transactions
- If it is ok for an operation to take place out of order, you can use `commute` instead of `alter` for a performance boost
- maybe I need to refer to Clojure for the brave and true for a nice example of `commute`